In [1]:
import sys
sys.path.append('../')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from ATARI.ModelData.particle_pair import Particle_Pair
from ATARI.ModelData.particle import Particle, Neutron

from ATARI.TAZ import ATARI_interface
from ATARI.TAZ.PTBayes import PTBayes
from ATARI.TAZ.RunMaster import RunMaster

In [2]:
# %matplotlib widget

In [ ]:
Ta181 = Particle(Z=73, A=181, I=3.5, mass=180.94803, name='Ta181')
Ta_pair = Particle_Pair(isotope = "Ta181",
                        resonance_ladder = pd.DataFrame(),
                        formalism = "XCT",
                        energy_range = [200,1000],
                        ac = 0.8127,
                        target=Ta181,
                        projectile=Neutron,
                        l_max = 1
)

# print quant number map up to l_max
Ta_pair.map_quantum_numbers(print_out=True)

# J = 3, <D> = 9.0030 eV, <gn2> = 452.56615 meV, <gg2> = 32.0 meV
# J = 4, <D> = 8.3031 eV, <gn2> = 332.24347 meV, <gg2> = 32.0 meV

# add spin group information for both s-wave resonances
Ta_pair.add_spin_group(Jpi='3.0',
                       J_ID=1,
                       D=9.0030,
                       gn2_avg=452.56615,
                       gn2_dof=1,
                       gg2_avg=32.0,
                       gg2_dof=1000)

Ta_pair.add_spin_group(Jpi='4.0',
                       J_ID=2,
                       D=8.3031,
                       gn2_avg=332.24347,
                       gn2_dof=1,
                       gg2_avg=32.0,
                       gg2_dof=1000)

print()
print("Interrogate spin group attribute:")
print(Ta_pair.spin_groups[3.0])


print()
print("Resonance ladder sampled from resonance statistics:")
print(Ta_pair.sample_resonance_ladder())

Spin group data for all parity
(Jpi, #Chs, l-waves)
(-5.0, 1, [1])
(-4.0, 2, [1, 1])
(-3.0, 2, [1, 1])
(-2.0, 1, [1])
(3.0, 1, [0])
(4.0, 1, [0])

Interrogate spin group attribute:
{'Jpi': 3.0, 'J_ID': 1, '<D>': 9.003, '<gn2>': 452.56615, 'n_dof': 1, '<gg2>': 32.0, 'g_dof': 1000, 'quantiles': {'D01': 1.0184333135415622, 'D99': 21.80043460630884, 'gn01': 0.0710926470659408, 'gn99': 3002.729610372257, 'gg01': 28.765198301747624, 'gg99': 35.42300781927096, 'gt01': 28.836290948813566, 'gt99': 3038.152618191528}, 'chs': 1, 'Ls': [0]}

Resonance ladder sampled from resonance statistics:
             E         Gg         Gn1  J_ID        gg2          gn2  Jpi    L
0   205.971832  64.283150  -33.273570   1.0  32.141575  -652.916082  3.0  0.0
1   209.931572  61.602819    5.064385   1.0  30.801410    98.435044  3.0  0.0
2   212.482400  63.559758  102.447500   1.0  31.779879  1979.255080  3.0  0.0
3   219.604130 -62.148497   26.024084   1.0 -31.074249   494.557840  3.0  0.0
4   227.657597 -61.114

In [4]:
# Ta_pair.resonance_ladder

reaction, res_ladder_TAZ, spin_ids = ATARI_interface.ATARI_to_TAZ(Ta_pair)
print(reaction)
print(res_ladder_TAZ)

In [ ]:
reaction, resonances, Spingroup_IDs = TAZ.ATARI_interface.ATARI_to_TAZ(Ta_pair, false_dens=0.0)

print('Target Particle:')
print(repr(reaction.targ))
print()
print('Reaction Parameters')
print(reaction)

### Porter-Thomas Probabilities

`PTBayes` calculates resonance width probabilities using Porter-Thomas distribution. This is almost always the first step in any TAZ algorithm.

In [ ]:
PT_probs, PT_log_likelihood = TAZ.PTBayes(resonances, reaction)
print('Porter-Thomas Posterior')
print(PT_probs)

### Preparing RunMaster
`RunMaster` is a wrapper around `Encore` that performs pre-and-post processing for each algorithm in `Encore`. Creating a `RunMaster` object will precalculate level-spacings and prepare the __cumulative probabilities__ which will be used in many of the `Encore` algorithms. Note that the `WigMaxLikelihoods` algorithm is a class method of `RunMaster` (and `Encore`) as it does not need to calculate __cumulative probabilities__.

`RunMaster` also performs __merging__ if specified which is an approximation that allows one to run `Encore` algorithms with less effective spingroups multiple times. This feature is only useful when 4 or more spingroups are provided. __Merging__ is only supported by `WigBayes` at this time.

In [7]:
distributions = reaction.distributions('Wigner')
runMaster = TAZ.RunMaster(Ta_pair.resonance_ladder['E'],
                          Ta_pair.energy_range,
                          level_spacing_dists=distributions,
                          false_dens=reaction.false_dens,
                          prior=PT_probs,
                          log_likelihood_prior=PT_log_likelihood)

### WigBayes
The WigBayes algorithm is run below. The `analysis` module holds various functions for testing performance.

In [8]:
PT_probs, PT_log_likelihood = PTBayes(res_ladder_TAZ, reaction)
print(PT_probs)

[[0.50644368 0.49355632 0.        ]
 [0.44144739 0.55855261 0.        ]
 [0.45113718 0.54886282 0.        ]
 [0.63563239 0.36436761 0.        ]
 [0.44253999 0.55746001 0.        ]
 [0.49060498 0.50939502 0.        ]
 [0.44687699 0.55312301 0.        ]
 [0.44282432 0.55717568 0.        ]
 [0.47378121 0.52621879 0.        ]
 [0.44438276 0.55561724 0.        ]
 [0.44509896 0.55490104 0.        ]
 [0.46044004 0.53955996 0.        ]
 [0.47486585 0.52513415 0.        ]
 [0.44183263 0.55816737 0.        ]
 [0.44845329 0.55154671 0.        ]
 [0.4426297  0.5573703  0.        ]
 [0.45684452 0.54315548 0.        ]
 [0.47633313 0.52366687 0.        ]
 [0.48985445 0.51014555 0.        ]
 [0.44226129 0.55773871 0.        ]
 [0.57272553 0.42727447 0.        ]
 [0.55009059 0.44990941 0.        ]
 [0.48393714 0.51606286 0.        ]
 [0.52778031 0.47221969 0.        ]
 [0.52293519 0.47706481 0.        ]
 [0.44147447 0.55852553 0.        ]
 [0.59428839 0.40571161 0.        ]
 [0.44299347 0.55700653 0.  

In [8]:
distributions = reaction.distributions('Wigner')
runMaster = RunMaster(res_ladder_TAZ['E'], reaction.EB, distributions, false_dens=reaction.false_dens, prior=PT_probs, log_likelihood_prior=PT_log_likelihood, verbose=True)
posterior = runMaster.WigBayes()

print()
print(posterior)

Finding level-spacing probabilities for group 0.
Finding level-spacing probabilities for group 1.
Creating ENCORE pipeline.
Finished ENCORE initialization.
Starting WigBayes
Finished running WigBayes

[[ 5.02782097e-01  4.97217906e-01 -2.79740897e-09]
 [ 4.61967946e-01  5.38032057e-01 -2.79740853e-09]
 [ 4.33983456e-01  5.66016546e-01 -2.79740897e-09]
 [ 6.49743909e-01  3.50256093e-01 -2.79740897e-09]
 [ 3.72177421e-01  6.27822582e-01 -2.79740897e-09]
 [ 5.28337062e-01  4.71662940e-01 -2.79740897e-09]
 [ 4.38921735e-01  5.61078267e-01 -2.79740875e-09]
 [ 4.56367156e-01  5.43632847e-01 -2.79740897e-09]
 [ 5.20879070e-01  4.79120933e-01 -2.79740897e-09]
 [ 4.38030917e-01  5.61969086e-01 -2.79740875e-09]
 [ 4.48169489e-01  5.51830514e-01 -2.79740875e-09]
 [ 4.89263515e-01  5.10736488e-01 -2.79740897e-09]
 [ 4.87092487e-01  5.12907510e-01  2.77127887e-09]
 [ 4.76041820e-01  5.23958178e-01  2.77260370e-09]
 [ 5.00609420e-01  4.99390577e-01  2.79432077e-09]
 [ 4.13296848e-01  5.86703149e-01 